# Agregação e classificação

Nesta seção, aprenderemos como resumir registros usando os operadores SQL `GROUP BY` e `ORDER BY`. Ao longo do caminho, aprenderemos funções de agregação como `SUM`, `COUNT`, `MIN`, `MAX` e `AVG`.

# Configuração

Baixe o banco de dados SQLite e inicialize uma conexão SQLite3.

In [ ]:
import sqlite3
import pandas as pd
import urllib.request

# baixe o banco de dados SQLite e conecte-se a ele
urllib.request.urlretrieve("https://github.com/thomasnield/anaconda_intro_to_sql/blob/main/company_operations.db?raw=true", "company_operations.db")
conn = sqlite3.connect('company_operations.db')

Continuaremos trabalhando com a tabela `WEATHER_MONITOR` e resumiremos os registros usando funções de agregação.

# Funções de agregação e GROUP BY

Vamos dar uma olhada em três campos na tabela `WEATHER_MONITOR`.

In [ ]:
sql = """

SELECT REPORT_CODE, REPORT_DATE, RAIN

FROM WEATHER_MONITOR 

"""

pd.read_sql(sql, conn)

Digamos que queremos encontrar o total de `RAIN` em toda a tabela. Se removermos os campos `REPORT_CODE` e `REPORT_DATE` e colocarmos `SUM()` em torno de `RAIN`, observe o que acontece.

In [ ]:
sql = """

SELECT SUM(RAIN) AS TOTAL_RAIN

FROM WEATHER_MONITOR 

"""

pd.read_sql(sql, conn)

Portanto, temos 1720,78 polegadas de chuva total em toda a tabela. Vamos dividir `TOTAL_RAIN` por `LOCATION_ID`. Podemos fazer isso selecionando `LOCATION_ID` e executando `GROUP BY` nele.

In [ ]:
sql = """

SELECT LOCATION_ID, SUM(RAIN) AS TOTAL_RAIN

FROM WEATHER_MONITOR 

GROUP BY LOCATION_ID

"""

pd.read_sql(sql, conn)

Observe como agora temos as somas divididas por `LOCATION_ID`, ou seja, somamos `TOTAL_CHUVA` por `LOCATION_ID`. Se quiséssemos obter o total por `LOCATION_ID` e `YEAR`, poderíamos dividi-lo por esses dois campos/expressões.

In [ ]:
sql = """

SELECT 
LOCATION_ID, 
strftime('%Y', REPORT_DATE) AS YEAR, 
SUM(RAIN) AS TOTAL_RAIN

FROM WEATHER_MONITOR 

GROUP BY LOCATION_ID, YEAR

"""

pd.read_sql(sql, conn)

Observe também que podemos usar `GROUP BY` com índice ordinal para cada coluna/expressão selecionada, em vez do nome da coluna. Observe que isso usa indexação de base 1.

In [ ]:
sql = """

SELECT 
LOCATION_ID, 
strftime('%Y', REPORT_DATE) AS YEAR, 
SUM(RAIN) AS TOTAL_RAIN

FROM WEATHER_MONITOR 

GROUP BY 1, 2

"""

pd.read_sql(sql, conn)

Existem outras funções de agregação além de `SUM()`. `MIN()` encontra o valor mínimo para uma determinada coluna, enquanto `MAX()` encontra o máximo. `AVG()` calcula a média da coluna, enquanto `COUNT()` conta o número de valores não nulos para essa coluna. Aqui estão todas as cinco funções de agregação para criar um relatório resumindo estatísticas descritivas de chuva por `LOCATION_ID` e `YEAR`.

In [ ]:
sql = """

SELECT 
LOCATION_ID, 
strftime('%Y', REPORT_DATE) AS YEAR, 

SUM(RAIN) AS TOTAL_RAIN, 
MIN(RAIN) AS MIN_RAIN,
MAX(RAIN) AS MAX_RAIN,
AVG(RAIN) AS AVG_RAIN, 
COUNT(RAIN) AS COUNT_RAIN

FROM WEATHER_MONITOR 

GROUP BY LOCATION_ID, YEAR

"""

pd.read_sql(sql, conn)

Também podemos usar um filtro `WHERE` para permitir que apenas determinados registros sejam qualificados em nossas agregações. Abaixo, calculamos o total de `RAIN` por `YEAR` e `LOCATION_ID`, mas apenas onde um `TORNADO` estava presente.

In [ ]:
sql = """

SELECT 
LOCATION_ID, 
strftime('%Y', REPORT_DATE) AS YEAR, 
SUM(RAIN) AS TOTAL_TORNADO_RAIN

FROM WEATHER_MONITOR 

WHERE TORNADO = 1
GROUP BY LOCATION_ID, YEAR

"""

pd.read_sql(sql, conn)

## Contagem de Registros

Se você quiser contar o número de registros em uma tabela, passe o registro inteiro para a função `COUNT()` em vez de um campo específico. Isso pode ser feito usando um asterisco `*`.

In [ ]:
sql = """

SELECT COUNT(*) AS RECORD_COUNT

FROM WEATHER_MONITOR 
"""

pd.read_sql(sql, conn)

Todas as outras operações que usamos anteriormente para segmentar e filtrar registros também podem ser usadas com `COUNT(*)`. Abaixo, dividimos a contagem de registros por `YEAR`, mas contamos apenas os registros em que `RAIN` tinha pelo menos 2 polegadas.

In [ ]:
sql = """

SELECT 
strftime('%Y', REPORT_DATE) AS YEAR, 

COUNT(*) AS RECORD_COUNT

FROM WEATHER_MONITOR 

WHERE RAIN >= 2

GROUP BY YEAR 
"""

pd.read_sql(sql, conn)

## Classificação

Vamos dar uma olhada na consulta abaixo mostrando o `TOTAL_RAIN` por `YEAR` e `MONTH`.

In [ ]:
sql = """

SELECT 
CAST(strftime('%Y', REPORT_DATE) AS INTEGER) AS YEAR, 
CAST(strftime('%m', REPORT_DATE) AS INTEGER) AS MONTH, 

SUM(RAIN) AS TOTAL_RAIN

FROM WEATHER_MONITOR 

GROUP BY YEAR, MONTH
"""

pd.read_sql(sql, conn)

Observe que os registros coincidentemente são ordenados por `YEAR` em ordem crescente e `MONTH` em ordem crescente. Você nunca deve esperar que os registros retornem em qualquer ordem sem um `ORDER BY`, mesmo que o mecanismo SQL tenha uma implementação que dê essa impressão. Isso pode acontecer especialmente se os dados forem armazenados fisicamente em uma ordem ordenada por (por exemplo, cronologicamente).

Para impor uma ordem crescente por `YEAR` e `MONTH`, adicione um operador `ORDER BY`.

In [ ]:
sql = """

SELECT 
CAST(strftime('%Y', REPORT_DATE) AS INTEGER) AS YEAR, 
CAST(strftime('%m', REPORT_DATE) AS INTEGER) AS MONTH, 

SUM(RAIN) AS TOTAL_RAIN

FROM WEATHER_MONITOR 

GROUP BY YEAR, MONTH

ORDER BY YEAR, MONTH
"""

pd.read_sql(sql, conn)

Você também pode referenciar as expressões selecionadas usando o índice ordinal.

In [ ]:
sql = """

SELECT 
CAST(strftime('%Y', REPORT_DATE) AS INTEGER) AS YEAR, 
CAST(strftime('%m', REPORT_DATE) AS INTEGER) AS MONTH, 

SUM(RAIN) AS TOTAL_RAIN

FROM WEATHER_MONITOR 

GROUP BY 1, 2

ORDER BY 1, 2
"""

pd.read_sql(sql, conn)

Se quisermos que os anos mais recentes sejam exibidos primeiro, adicione a palavra-chave `DESC` para classificar um determinado campo em ordem decrescente.

In [ ]:
sql = """

SELECT 
CAST(strftime('%Y', REPORT_DATE) AS INTEGER) AS YEAR, 
CAST(strftime('%m', REPORT_DATE) AS INTEGER) AS MONTH, 

SUM(RAIN) AS TOTAL_RAIN

FROM WEATHER_MONITOR 

GROUP BY YEAR, MONTH

ORDER BY YEAR DESC, MONTH
"""

pd.read_sql(sql, conn)

## EXERCÍCIO

Complete a consulta abaixo para encontrar a queda de neve total, mínima e máxima por ano. Ordene o ano em ordem decrescente para que o ano mais recente fique no topo.

In [ ]:
sql = """

SELECT 
strftime('%Y', REPORT_DATE) AS YEAR, 

? AS TOTAL_SNOW, 
? AS MIN_SNOW,
? AS MAX_SNOW

FROM WEATHER_MONITOR 

? BY ?
? BY ? DESC
"""

pd.read_sql(sql, conn)

### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
sql = """

SELECT 
strftime('%Y', REPORT_DATE) AS YEAR, 

SUM(SNOW) AS TOTAL_SNOW, 
MIN(SNOW) AS MIN_SNOW,
MAX(SNOW) AS MAX_SNOW

FROM WEATHER_MONITOR 

GROUP BY YEAR
ORDER BY YEAR DESC
"""

pd.read_sql(sql, conn)